In [91]:
import xmltodict
import json
import pathlib
import os

def word_or_char(val):
    lv = len(val)
    if lv>1:
        return "word"
    else:
        return "char"
    

base_mockup ={
    'image':{
        'filename':"",
        'dim':{
            "width":0,
            "height":0,
            "channel":0
        }  
    },
    "object":[]
}

object_mockup = {
    "class_type": "",
    "class_value": "",
    "coord_type": "",
    "coord_value": {
      "xmin":0,
      "xmax":0,
      "ymin":0,
      "ymax":0
    }
}

def open_xmlfile_to_dict(filepath):
    with open(xmlfile) as fd:
        doc = xmltodict.parse(fd.read())
        xml_dict = doc['annotation']
    return xml_dict

def save_to_file(jsonfile, content):
    with open(jsonfile, 'w+') as json_file:
        json.dump(content, json_file, indent=4)


def convert_xml_to_dict(xmldict):
    conv_json = base_mockup.copy()
    conv_json['image']['filename'] = xmldict['filename']
    conv_json['image']['dim']['width'] = int(xmldict['size']['width'])
    conv_json['image']['dim']['height'] = int(xmldict['size']['height'])
    conv_json['image']['dim']['channel'] = int(xmldict['size']['depth'])

    for obj in xmldict['object']:
        object_json = object_mockup.copy()
        bndbox = obj['bndbox']  

        object_json['class_type'] = word_or_char(obj['name'])
        object_json['class_value'] = obj['name']
        object_json['coord_type'] = 'bndbox'
        object_json['coord_value']['xmin'] = int(bndbox['xmin'])
        object_json['coord_value']['ymin'] = int(bndbox['ymin'])
        object_json['coord_value']['xmax'] = int(bndbox['xmax'])
        object_json['coord_value']['ymax'] = int(bndbox['ymax'])

        conv_json['object'].append(object_json)
    
    return conv_json 

def convert(xmlfile, jsonfile):
    print(f'Load file xml in path: {xmlfile}')
    xml_dict = open_xmlfile_to_dict(xmlfile)

    conv_json = convert_xml_to_dict(xml_dict)
    
    save_to_file(jsonfile, content=conv_json)
    print(f'Convert xml to json in path: {jsonfile}')
    
    
    
def convert_xml_to_json(path):

    base_path = pathlib.Path(path)
    list_files = base_path.glob('*.xml')
    
    for path_file in list(list_files):
        base_path = path_file.parent
        xml_file = path_file.name
        xml_filename = os.path.join(str(base_path), str(xml_file))

        json_file = path_file.stem + ".json"
        json_filename = os.path.join(str(base_path), str(json_file))
        
        
        convert(xml_filename, json_filename)

In [122]:
path = '/home/abbiyanaila/Tigapilar/CRAFT bounding box/Testing/Dataset Class B/'
convert_xml_to_json(path)